In [1]:
for var in list(globals()):
    if var != "__name__":
        del globals()[var]
        
import pandas as pd
import json
import os
from tqdm import tqdm
import gzip
from global_variables import *

In [3]:
date_start = '2021-01-01'
date_end = '2022-12-01'

file_suffix = date_start.replace('-', '')+"_"+date_end.replace('-', '')
print(file_suffix)

20210101_20221201


In [5]:
# read_monthly_ft_data.ipynb creates this file
ft_data_path = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/filtered_ft_data_'+file_suffix+'.csv.gz'
ft = pd.read_csv(ft_data_path,compression='gzip')

C:\Users\dratnadiwakara2\AppData\Local\Temp\ipykernel_15460\961726700.py:3: DtypeWarning: Columns (18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  ft = pd.read_csv(ft_data_path,compression='gzip')


In [6]:
visits_file = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/cbgs_place_month_visits_count'+file_suffix+'.csv'
visit_summary_file = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/place_month_visit_summary'+file_suffix+'.csv'

In [7]:

header_option_visits_file = True  
header_option_visit_summary_file = True 

with tqdm(total=len(ft)) as pbar:
    
    for index in range(0, len(ft)):
        
        temp = ft.loc[index]
        
        data = json.loads(temp['VISITOR_HOME_CBGS'])
        df1 = pd.DataFrame(data.items(), columns=['CBGS', 'no_visits'])
        df1['PLACEKEY'] = temp['PLACEKEY']
        df1['DATE_RANGE_START'] = temp['DATE_RANGE_START']
        
        df2 = temp[['PLACEKEY','DATE_RANGE_START','RAW_VISIT_COUNTS','RAW_VISITOR_COUNTS','DISTANCE_FROM_HOME','MEDIAN_DWELL']]
        df2['android'] = 0
        df2['ios'] = 0
        
        try:
            dt = json.loads(temp['DEVICE_TYPE'])
            df2['android'] = dt['android']
        except:
            pass
        
        try:
            dt = json.loads(temp['DEVICE_TYPE'])
            df2['ios'] = dt['ios']
        except:
            pass
        df2_transposed = df2.to_frame().reset_index().T
        df2_transposed.columns = df2_transposed.iloc[0]
        df2_transposed = df2_transposed[1:]
        
        if os.path.isfile(visits_file):
            header_option_visits_file = False  
        if os.path.isfile(visit_summary_file):
            header_option_visit_summary_file = False  
    
        df1.to_csv(visits_file, index=False, mode='a', header=header_option_visits_file,sep="|")
        df2_transposed.to_csv(visit_summary_file, index=False, mode='a', header=header_option_visit_summary_file,sep="|")
        
        pbar.update(1)

100%|██████████| 1507725/1507725 [2:46:43<00:00, 150.71it/s]  


In [8]:
visits = pd.read_csv(visits_file)
visits_summary = pd.read_csv(visit_summary_file)

In [9]:
with gzip.open(visits_file+'.gz', 'wt', encoding='utf-8') as gzipped_file:
    visits.to_csv(gzipped_file, index=False)

with gzip.open(visit_summary_file+'.gz', 'wt', encoding='utf-8') as gzipped_file:
    visits_summary.to_csv(gzipped_file, index=False)